In [1]:
import pandas as pd 
import pickle
import json
import os
import os.path
import numpy as np
import seaborn as sns
from collections import defaultdict
import joblib
import matplotlib.pyplot as plt
from matplotlib.ticker import (
                               FormatStrFormatter, 
                               AutoMinorLocator,
                               FuncFormatter,
                               )
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
%matplotlib inline
from scipy.spatial import distance
import math


import os
os.chdir('G:\My Drive\MSc_project\.MAIN')


c:\Users\berke\anaconda3\envs\env-pytorch\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\berke\anaconda3\envs\env-pytorch\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\berke\anaconda3\envs\env-pytorch\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:

hashtags = ['avengers','borisjohnson','blm','brexit','climatechange','covid','gaza','loveisland','monkeypox','nhs','olivianewtonjohn','supercup','UkraineWar']


for h_i,hashtag in enumerate(hashtags):

    cats = ['sentiment', 'topic', 'topic_single', 'irony', 'emoji', 'offensive','gender']

    df = pd.read_csv(f'multisource_analysis/data/{hashtag}_scored_tweets.csv')
    df.drop(['Unnamed: 0', 'text','hashtag','user_id']+cats, inplace = True, axis =1)

    # readability statistics are nan when the tweet contains no text. so change these scores to 0
    reads = ['ARI','LIX','RIX','complex_words','Coleman-Liau', 'FleschReadingEase', 'GunningFogIndex', 'LIX', 'SMOGIndex', 'RIX', 'DaleChallIndex']
    for r in reads:
        df[[r]] = df[[r]].fillna(value=0).copy()

    cases = list(set(df['target'].tolist()))

    infectors = df[df['is-target']==1]
    informers = df[df['is-informer']==1]

    n = len(infectors)

    # clean a little
    for sub in [infectors,informers]:
        sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)

    cols = infectors.columns.tolist()

    best_formers = []
    av_formers = []

    for case_id in cases:

        o_fect = infectors.copy()[infectors['target']==case_id]
        fect = o_fect.copy().drop('target', axis=1).to_numpy()
        o_formers = informers.copy()[informers['target']==case_id]
        formers = o_formers.copy().drop('target', axis=1).to_numpy()

        av_former = informers.copy()[informers['target']==case_id]
        av_former = av_former.copy().drop('target', axis=1).mean().to_numpy()

        # get the closest feature to the target per feature
        best_calc = abs(fect - formers)
        best_idx = [ np.argmin(abs(row.astype(float))) for row in best_calc.T ]
        best_former = np.array( [form[best_idx[i]] for i,form in enumerate(formers.T) ] ) # we keeep the informer with the closer scores per feature
        
        av_formers.append(av_former.tolist())
        best_formers.append(best_former.tolist())

    cols.remove('target')

    infectors.drop('target', inplace = True, axis =1)
    av_informers_df = pd.DataFrame(av_formers, columns = cols)
    best_informers_df = pd.DataFrame(best_formers, columns = cols)

    infectors.insert(0,column = 'hashtag', value= [hashtag]*n )
    av_informers_df.insert(0,column = 'hashtag', value= [hashtag]*n )
    best_informers_df.insert(0,column = 'hashtag', value= [hashtag]*n )
    

    if h_i == 0:
        all_infector_df = infectors
        all_av_informer_df = av_informers_df
        all_best_informer_df = best_informers_df
    else:   
        all_infector_df = pd.concat( [all_infector_df, infectors ], axis = 0 )
        all_av_informer_df = pd.concat( [all_av_informer_df, av_informers_df ], axis = 0 )
        all_best_informer_df = pd.concat( [all_best_informer_df, best_informers_df ], axis = 0 )
    

    

    print(f'\n\nAdded {hashtag} to DATAFRAME\n----------------------------------------------------------------')

C:\Users\berke\AppData\Local\Temp\ipykernel_37268\361570906.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_37268\361570906.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_37268\361570906.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user



Added avengers to DATAFRAME
----------------------------------------------------------------


C:\Users\berke\AppData\Local\Temp\ipykernel_37268\361570906.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_37268\361570906.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_37268\361570906.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user



Added borisjohnson to DATAFRAME
----------------------------------------------------------------


C:\Users\berke\AppData\Local\Temp\ipykernel_37268\361570906.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_37268\361570906.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_37268\361570906.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

...

C:\Users\berke\AppData\Local\Temp\ipykernel_37268\361570906.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_37268\361570906.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_37268\361570906.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user



Added supercup to DATAFRAME
----------------------------------------------------------------


C:\Users\berke\AppData\Local\Temp\ipykernel_37268\361570906.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_37268\361570906.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)




Added UkraineWar to DATAFRAME
----------------------------------------------------------------


C:\Users\berke\AppData\Local\Temp\ipykernel_37268\361570906.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  infectors.drop('target', inplace = True, axis =1)


### SINGLE COLS

In [56]:
from scipy.stats import ks_2samp

av_scores = []
best_scores = []

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in np.arange(0, len(lst), n):
        yield lst[i:i + n]

def count_range_in_list(li, min, max):
	ctr = 0
	for x in li:
		if min <= x <= max:
			ctr += 1
	return ctr

bin_size = 15
n_cols = [ f'Bin {i}' for i in range(bin_size)]

for i,col in enumerate(cols):

    fec = all_infector_df[col].tolist()
    av_form = all_av_informer_df[col].tolist()
    best_form = all_best_informer_df[col].tolist()

    diffs = abs(np.array(fec)- np.array(best_form))
    diffs = diffs*100

    low = min(diffs)
    hi = max(diffs)

    # rng = list( chunks(np.arange(low,hi), 25) )
    

    rng = np.array_split(np.arange(low,hi), bin_size)

    diff_bins = [ count_range_in_list(diffs, r[0], r[-1]) for r in rng]

    # df[col] = diff_bins

    if i ==0:
        df = pd.DataFrame(diff_bins, columns = [col]).T
        df.columns = n_cols
    else:
        df.loc[col] = diff_bins


    # plt.hist(diffs, density=False, bins = 15, edgecolor='black')
    # plt.ylabel('Frequency of occurence')
    # plt.xlabel('Bin')
    # plt.title(f'Infector and Informer Difference of feature: {col}')
    # plt.figure(i)
    
    # ax = sns.histplot(data=diffs, bins=bin_size, kde = False)

df.to_csv('multisource_analysis/difference_scores/tweet_diffs.csv')

### GROUPED COLS